###**Apache Spark DataFrames Assignment  Project**###

##**Problem statement**##

As a Data professional, you need to perform an analysis by answering questions about
some stock market data on Safaricom from the years 2012-2017

###**Required actions**
**Data Importation and Exploration**:

● Start a spark session and load the stock file while inferring the data types.

● Determine the column names

● Make observations about the schema.

● Show the first 5 rows

● Use the describe method to learn about the data frame

**Data Preparation**

● Format all the data to 2 decimal places i.e. format_number()

● Create a new data frame with a column called HV Ratio that is the ratio of the
High Price versus volume of stock traded for a day

**Data Analysis**

● What day had the Peak High in Price?

● What is the mean of the Close column?

● What is the max and min of the Volume column?

● How many days was the Close lower than 60 dollars?

● What percentage of the time was the High greater than 80 dollars?

● What is the Pearson correlation between High and Volume?

● What is the max High per year?

● What is the average Close for each Calendar Month?

**Data Source** :Dataset URL (CSV File): https://bit.ly/3pmchka

##**Pre-requisites**##

In [1]:
#pyspark instalation done 
#
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 27.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=5d7cf4c691cb37d38c4494071464269a6274b8467a47efe8cfff9a7ba9fc6156
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [29]:
#then run the spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
from pyspark.sql import SQLContext

In [22]:
# Starting a spark session and loading the stock file while inferring the data types

stock_saf_df = spark.read.csv("saf_stock.csv", header=True, inferSchema=True)

In [23]:
# Determining the column names

print("Column Names:", stock_saf_df.columns)

Column Names: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


In [24]:
# Making observations about the schema
print("Schema:"), stock_saf_df.printSchema()

Schema:
root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



(None, None)

In [25]:
# Showing the first 5 rows of our data
stock_saf_df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [26]:
# to learn about the data frame we use the describe method as below 

stock_saf_df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

##Performing Data Preparation##

In [27]:
# Formating all the data to 2 decimal places i.e. format_number()

from pyspark.sql.functions import format_number

##
numeric_cols = [col[0] for col in stock_saf_df.dtypes if col[1] in ['double', 'int']]
stock_saf_df = stock_saf_df.select(stock_saf_df["Date"], *(format_number(col, 2).alias(col) for col in numeric_cols))
stock_saf_df.show(5)

+-------------------+-----+-----+-----+-----+-------------+---------+
|               Date| Open| High|  Low|Close|       Volume|Adj Close|
+-------------------+-----+-----+-----+-----+-------------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12,668,800.00|    52.62|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9,593,300.00|    52.08|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12,768,200.00|    51.83|
|2012-01-06 00:00:00|59.42|59.45|58.87|59.00| 8,069,400.00|    51.46|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6,679,300.00|    51.62|
+-------------------+-----+-----+-----+-----+-------------+---------+
only showing top 5 rows



In [28]:
#removing the commas from the the Volume column and creating new HV ratio column(ratio of High Price/Vlume of Stock traded)
from pyspark.sql.functions import regexp_replace, col
stock_saf_df = stock_saf_df.withColumn("Volume", regexp_replace(col("Volume"), ",", ""))
stock_saf_df = stock_saf_df.withColumn("HV Ratio", stock_saf_df.High/ stock_saf_df.Volume)
stock_saf_df.show(5)

+-------------------+-----+-----+-----+-----+-----------+---------+--------------------+
|               Date| Open| High|  Low|Close|     Volume|Adj Close|            HV Ratio|
+-------------------+-----+-----+-----+-----+-----------+---------+--------------------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800.00|    52.62|4.819714574387472E-6|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300.00|    52.08|6.290848821573389...|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200.00|    51.83|4.669413073103491E-6|
|2012-01-06 00:00:00|59.42|59.45|58.87|59.00| 8069400.00|    51.46|7.367338339901356E-6|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300.00|    51.62|8.915604928660188E-6|
+-------------------+-----+-----+-----+-----+-----------+---------+--------------------+
only showing top 5 rows



###**Performing Data Analysis**###

In [31]:
# sql context created and getting list of tables names in the sql context
sqlCtx = SQLContext(spark)
stock_saf_df.createOrReplaceTempView("stock_data")
print("Tables in SQLContext:", sqlCtx.tableNames())

Tables in SQLContext: ['stock_data']


In [32]:
#day with peak high price is obtained using SQL

peak_high_price = sqlCtx.sql("SELECT Date, High FROM stock_data ORDER BY High DESC LIMIT 1")
peak_high_price.show()

+-------------------+-----+
|               Date| High|
+-------------------+-----+
|2015-01-13 00:00:00|90.97|
+-------------------+-----+



In [33]:
# to find the mean of the close column 

stock_saf_df[['close']].describe().show()

+-------+-----------------+
|summary|            close|
+-------+-----------------+
|  count|             1258|
|   mean|72.38844992050863|
| stddev|6.756859155425468|
|    min|            56.42|
|    max|            90.47|
+-------+-----------------+



the mean of the close column is :72.39

In [34]:
#getting the min/max values of Volume column
stock_saf_df[['Volume']].describe().show()

+-------+-----------------+
|summary|           Volume|
+-------+-----------------+
|  count|             1258|
|   mean|8222093.481717011|
| stddev|  4519780.8431556|
|    min|      10010500.00|
|    max|       9994400.00|
+-------+-----------------+



min value is 10010500.00 and max value is 9994400.00

In [35]:
#counting the number of days with close price < 60

days_60 = stock_saf_df[stock_saf_df['close'] < 60]
days_60.count()


81

In [44]:
#getting % time where High is >80$
high_greater = sqlCtx.sql("SELECT COUNT(*) FROM stock_data WHERE High > 80")
high_count = high_greater.collect()[0][0]
high_greater = sqlCtx.sql("SELECT COUNT(*) FROM stock_data")
count = high_greater.collect()[0][0]

# finding and converting the results to percentage
percentage = (high_count/count) * 100

print("% time where High is >80$: {:.2f}%".format(percentage))

% time where High is >80$: 8.43%


In [46]:
#finding the pearson correlation between High and low
# to get this right we need to import correlation functon frm pyspark

from pyspark.sql.functions import corr
high_greater = stock_saf_df.select(corr("High", "Volume").alias("correlation")).collect()[0][0]
print("Pearson correlation between High and Volume is:", high_greater)

Pearson correlation between High and Volume is: -0.33843260582148915


In [47]:
# to find the maximum High per Year, we group by year on SQL

high_max = sqlCtx.sql("SELECT year(Date) as year, max(High) as max_high FROM stock_data GROUP BY year(Date) ORDER BY max_high DESC")

# Show the result
high_max.show()

+----+--------+
|year|max_high|
+----+--------+
|2015|   90.97|
|2014|   88.09|
|2013|   81.37|
|2012|   77.60|
|2016|   75.19|
+----+--------+

